In [1]:
import cv2

import numpy as np

import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu

import scipy
from scipy.signal import convolve

import os

import xml.etree.ElementTree as ET

import math
from decimal import *

In [2]:
def unsharp_mask(img, intensity):
    '''
    np.array, float -> np.array
    Realça as bordas através da aplicação do unsharp mask 
    (convolução do filtro identidade - filtro laplaciano ponderado).
    
    Parâmetros
    ----------
    img : imagem de entrada a ser realçada pela técnica.
    intensity: intensidade da ponderação do realce de bordas.
    
    Return
    ------
    Numpy array contendo a imagem com bordas destacadas através do unsharp mask.
    '''
    kernel = np.array([[0,             -1*intensity,              0],
                       [-1*intensity,  (4*intensity)+1, -1*intensity],
                       [0,             -1*intensity,              0]])
    return convolve(img, kernel, mode='same')

In [3]:
def resultados(TP, FP, FN):
    '''
    int, int, int -> float, float, float
    Calcula as métricas de avaliação do método, Precisão, Revocação e Medida-F1.
    
    
    Parâmetros
    ----------
    TP: True positives, detecções que de fato pertencem ao quadro.
    FP: False positives, detecções que não pertencem ao quadro, de fato.
    FN: False negatives, células anotadas que não foram detectadas.
    
    Return
    ------
    Precisão: # de Acertos / # de Acertos + # de Erros
    Revocação: # de Acertos / #de ACertos + # de Omissões
    Medida-F1: 2 / (1/Precisão) + (1/Revocação)
    '''
    precisao = Decimal(TP) / Decimal(TP+FP)
    revocacao = Decimal(TP) / Decimal(TP+FN)
    f1 = Decimal(2) / Decimal(Decimal(1)/Decimal(precisao) + Decimal(1)/Decimal(revocacao))
    
    
    return precisao, revocacao, f1

In [4]:
def read_content(xml_file):
    '''
    arquivo.xml -> string, list
    Realiza o parsing de um arquivo xml contendo as anotações ground truth de um quadro do vídeo
    e retorna o nome do quadro e a lista de coordenadas das caixas que contém as células anotadas.
    
    Parâmetros
    ----------
    xml_file: caminho para o arquivo xml
    
    Return
    ------
    filename: string com o nome do arquivo contendo o frame.
    list_with_all_boxes: literalmente uma lista contendo as coordenadas dos limites de todas as caixas 
    que contém as células anotadas naquele quadro.
    '''
    
    #Define a árvore, e a raiz da mesma, que estrutura o arquivo
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    #Inicializa uma lista vazia
    list_with_all_boxes = []
    list_with_all_centers = []

    #Faz a leitura e o parsing do arquivo
    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)
        
        #Formata e adiciona a lista um conjunto de coordenadas correspondente a uma célula anotada
        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)
        list_with_all_centers.append(coord_to_center(xmin, ymin, xmax, ymax))

    return filename, list_with_all_boxes, list_with_all_centers

In [5]:
def coord_to_center(xmin, ymin, xmax, ymax):
    '''
    int, int, int, int -> (int,int)
    Dadas as coordendas dos vértices opostos de um quadrilátero, retorna as coordenadas do centro do mesmo.
   
    Parâmetros
    ----------
    xmin, ymin, xmax, ymax: coordenadas dos ponto Pmax e Pmin de um quadriláterio.
    
    Return
    ------
    Tupla de valores correspondentes a coordenada do centro do quadrilátero.
    '''
    return ((xmin+xmax)//2),((ymin+ymax)//2)

In [6]:
def min_dist(center,boxes):
    '''
    (int,int) , list -> int
    Dadas as coordenadas do centro de um circulo, e uma lista com todas as células anotadas em um frame, retorna
    a menor distância entre este ponto central do circulo e os centros das células anotadas representadas na lista
   
    Parâmetros
    ----------
    center: coordendas do centro do circulo
    boxes: lista de coordenadas dos quadriláteros contendo as células anotadas em um quadro.
    
    Return
    ------
    min(dist): distancia minima entre as células anotadas e o centro do circulo dado.
    '''
    dist = []
    for box in boxes:
        dist.append(scipy.spatial.distance.euclidean(center, coord_to_center(box[0],box[1],box[2],box[3])))
    return min(dist)

In [7]:
def min_dist_box(box, circles):
    '''
    (int, int, int, int) , list -> int
    Dadas as coordenadas dos vértices oposto de um quadrilátero, e uma lista com todas as coordenadas dos circulos
    detectados em um frame, retorna a menor distância entre este o centro do quadrilátero e os circulos contidos
    na lista.
   
    Parâmetros
    ----------
    box: coordendas dos vértices opostos do quadrilátero com a célula anotada
    circles: lista de coordenadas dos centros dos círculos detectados em um quadro.
    
    Return
    ------
    min(dist): distancia minima entre as células anotadas e o centro do circulo dado.
    '''
    dist = []
    for circle in circles:
        dist.append(scipy.spatial.distance.euclidean(coord_to_center(box[0],box[1],box[2],box[3]), (circle[0],circle[1])))
    return min(dist)

In [8]:
def TP_FP_FN(diretorio_img, diretorio_xml,mask):   
    '''
    diretorio, diretorio, numpy.array -> int, int, int
   
    Dados um diretorio de imagens, um diretorio de xml's, e uma máscara, realiza a aplicação da transformada
    de Hough para circulos em todas as imagens do diretório, e compara os resultados encontrados com as anotações
    contidas nos arquivos xml dentro do diretorio de xml's.
    Parâmetros
    ----------
    diretorio_img: Caminho do diretório populado com imagens a processar.
    diretorio_xml: Caminho do diretório populado com xml's das anotações ground truth respectiva aos quadros do vídeo
    mask: np.array contendo a máscara de segmentação da área de interesse do quadro.
    
    Return
    ------
    TP: True positives, detecções que de fato pertencem ao quadro.
    FP: False positives, detecções que não pertencem ao quadro, de fato.
    FN: False negatives, células anotadas que não foram detectadas.
    '''
        
    #Variavel local
    TP = 0
    FP = 0
    FN = 0
    
    #Lista todos os arquivos nos diretórios
    files_img = os.listdir(diretorio_img)   
    files_xml = os.listdir(diretorio_xml)

    #Organiza todos os arquivos encontrados
    files_img = np.sort(files_img)
    files_xml = np.sort(files_xml)

    #Extrai, uma-a-uma, as imagens do diretório para serem processadas 
    num_files_img = len(files_img)
    img_matrix = np.zeros((num_files_img,420,592), dtype=np.uint8)

    for file_index, file in enumerate(files_img):
        TP_frame = 0
        FP_frame = 0
        FN_frame = 0
        
        img_matrix[file_index] = cv2.imread((diretorio_img+file),0)

        #Conversão para RGB, para sobrepor os círculos encontrados
        cimg = cv2.cvtColor(img_matrix[file_index],cv2.COLOR_GRAY2BGR)
    
        #Aplicação do unsharp mask para o destaque de bordas
        img_enhanced = unsharp_mask(img_matrix[file_index], 1)
   
        #Aplicação da máscara
        img_enhanced_masked = np.multiply(img_enhanced,mask)
    
        #Conversão para uint8, requisito do método
        img_enhanced_masked_uint8 = np.uint8(img_enhanced_masked)
    
        #Definição dos parâmetros 
        par_dp = 1        #The inverse ratio of resolution || Resolução da matriz acumuladora 
        par_min_dist = 8  #Minimum distance between detected centers || Mínima distância entre os circulos detectados
        par_param1 = 70   #Upper threshold for the internal Canny edge detector || Limite superior para o detector de bordas interno 
        par_param2 = 8    #Threshold for center detection. || Rigidez do votador responsável pelo julgamento dos candidatos a círculos detectados
        par_minRadius = 4 #mMinimum radio to be detected. If unknown, put zero as default. || Raio de busca mínimo
        par_maxRadius = 9 #Maximum radius to be detected. If unknown, put zero as default || Raio de busca máximo
    
        #Aplicação da Técnica
        circles = cv2.HoughCircles(img_enhanced_masked_uint8, #Input image (grayscale, uint8)
                               cv2.HOUGH_GRADIENT,            #CV_HOUGH_GRADIENT: Define the detection method. Currently this is the only one available in OpenCV
                                par_dp,par_min_dist,param1=par_param1,param2=par_param2,minRadius=par_minRadius,maxRadius=par_maxRadius)
        
        circles_centers = circles[:,:,0:2]
        
        if len(circles) > 0:
            #Extraindo as anotações de cada frame
            name, boxes, boxes_centers = read_content("annotations_to_process/brain_1__{}.xml".format(file_index+1))

            #Buscando, para cada circulo detectado, um circulo anotado no mesmo lugar, se houver, TP++, se não, FP++
            for detected_circle in circles[0,:]:
                center = (detected_circle[0],detected_circle[1])
                if(min_dist(center,boxes) <= 7):
                    TP += 1
                    TP_frame +=1
                else:
                    FP += 1
                    FP_frame += 1

            #Para cada anotação, busca-se na lista de circulos por um com distância menor que 2, se não houver, FN++
            for box in boxes:
                if(min_dist_box(box,circles[0,:]) > 7):
                    FN += 1
                    FN_frame += 1

            #print ("Frame {}: True positives = {} || False positives = {} || False negatives = {}.".format(file_index,TP_frame, FP_frame, FN_frame))
   
    return TP,FP,FN

In [9]:
#Carregando a Máscara
mask = cv2.imread('maskt1.png',0)

#Extraindo Métricas
TP, FP, FN = TP_FP_FN("to_process/","annotations_to_process/",mask)

#Exibindo resultados
#print resultados(TP,FP,FN)[0]*100+resultados(TP,FP,FN)[1]*100+resultados(TP,FP,FN)[2]*100
print "True Positives: {}, False Positives: {}, False Negatives: {}".format(TP,FP,FN)
print "Precisão: {:.2f}%, Revocação: {:.2f}%, Medida F1 = {:.2f}%.".format(resultados(TP,FP,FN)[0]*100,resultados(TP,FP,FN)[1]*100,resultados(TP,FP,FN)[2]*100)

True Positives: 2711, False Positives: 2227, False Negatives: 2883
Precisão: 54.90%, Revocação: 48.46%, Medida F1 = 51.48%.


# Otimização dos Parâmetros:

In [37]:
def Test(arg_dp, arg_md, arg_par1, arg_par2, arg_minR, arg_maxR):   
    #Variavel local
    TP = 0
    FP = 0
    FN = 0
    mask = cv2.imread('maskt1.png',0)
    diretorio_img = "to_process/"
    diretorio_xml = "annotations_to_process/"
    #Lista todos os arquivos nos diretórios
    files_img = os.listdir("to_process/")   
    files_xml = os.listdir("annotations_to_process/")

    #Organiza todos os arquivos encontrados
    files_img = np.sort(files_img)
    files_xml = np.sort(files_xml)

    #Extrai, uma-a-uma, as imagens do diretório para serem processadas 
    num_files_img = len(files_img)
    img_matrix = np.zeros((num_files_img,420,592), dtype=np.uint8)

    for file_index, file in enumerate(files_img):
        TP_frame = 0
        FP_frame = 0
        FN_frame = 0
        
        img_matrix[file_index] = cv2.imread((diretorio_img+file),0)

        #Conversão para RGB, para sobrepor os círculos encontrados
        cimg = cv2.cvtColor(img_matrix[file_index],cv2.COLOR_GRAY2BGR)
    
        #Aplicação do unsharp mask para o destaque de bordas
        img_enhanced = unsharp_mask(img_matrix[file_index], 1)
   
        #Aplicação da máscara
        img_enhanced_masked = np.multiply(img_enhanced,mask)
    
        #Conversão para uint8, requisito do método
        img_enhanced_masked_uint8 = np.uint8(img_enhanced_masked)
    
        #Definição dos parâmetros 
        par_dp = arg_dp          #The inverse ratio of resolution || Resolução da matriz acumuladora 
        par_min_dist = arg_md    #Minimum distance between detected centers || Mínima distância entre os circulos detectados
        par_param1 = arg_par1    #Upper threshold for the internal Canny edge detector || Limite superior para o detector de bordas interno 
        par_param2 = arg_par2    #Threshold for center detection. || Rigidez do votador responsável pelo julgamento dos candidatos a círculos detectados
        par_minRadius = arg_minR #mMinimum radio to be detected. If unknown, put zero as default. || Raio de busca mínimo
        par_maxRadius = arg_maxR #Maximum radius to be detected. If unknown, put zero as default || Raio de busca máximo
    
        #Aplicação da Técnica
        circles = cv2.HoughCircles(img_enhanced_masked_uint8, #Input image (grayscale, uint8)
                               cv2.HOUGH_GRADIENT,            #CV_HOUGH_GRADIENT: Define the detection method. Currently this is the only one available in OpenCV
                                par_dp,par_min_dist,param1=par_param1,param2=par_param2,minRadius=par_minRadius,maxRadius=par_maxRadius)
        
        circles_centers = circles[:,:,0:2]
        
        if len(circles) > 0:
            #Extraindo as anotações de cada frame
            name, boxes, boxes_centers = read_content("annotations_to_process/brain_1__{}.xml".format(file_index+1))

            #Buscando, para cada circulo detectado, um circulo anotado no mesmo lugar, se houver, TP++, se não, FP++
            for detected_circle in circles[0,:]:
                center = (detected_circle[0],detected_circle[1])
                if(min_dist(center,boxes) <= 7):
                    TP += 1
                    TP_frame +=1
                else:
                    FP += 1
                    FP_frame += 1

            #Para cada anotação, busca-se na lista de circulos por um com distância menor que 2, se não houver, FN++
            for box in boxes:
                if(min_dist_box(box,circles[0,:]) > 7):
                    FN += 1
                    FN_frame += 1

            #print ("Frame {}: True positives = {} || False positives = {} || False negatives = {}.".format(file_index,TP_frame, FP_frame, FN_frame))
   
    #Exibindo resultados
    #print resultados(TP,FP,FN)[0]*100+resultados(TP,FP,FN)[1]*100+resultados(TP,FP,FN)[2]*100
    print ""
    print "TP: {}, FP: {}, FN: {}".format(TP,FP,FN)
    print "Pr: {:.2f}%, Re: {:.2f}%, F1 = {:.2f}%.".format(resultados(TP,FP,FN)[0]*100,resultados(TP,FP,FN)[1]*100,resultados(TP,FP,FN)[2]*100)
    print ""
    return 

In [33]:
#Extraindo Métricas

#Hardness = 6
Test(1, 1, 70, 6, 4, 9)
#Hardness = 7
Test(1, 1, 70, 7, 4, 9)
#Hardness = 8
Test(1, 1, 70, 8, 4, 9)
#Hardness = 9
Test(1, 1, 70, 9, 4, 9)
#Hardness = 10
Test(1, 1, 70, 10, 4, 9)

TP: 17399, FP: 19662, FN: 1425
Pr: 46.95%, Re: 92.43%, F1 = 62.27%.

TP: 11536, FP: 9117, FN: 1979
Pr: 55.86%, Re: 85.36%, F1 = 67.53%.

TP: 7598, FP: 4342, FN: 2642
Pr: 63.63%, Re: 74.20%, F1 = 68.51%.

TP: 5010, FP: 2141, FN: 3325
Pr: 70.06%, Re: 60.11%, F1 = 64.70%.

TP: 3270, FP: 1070, FN: 3990
Pr: 75.35%, Re: 45.04%, F1 = 56.38%.



In [43]:
#Hardness = 8
Test(1, 1, 70, 8, 3, 7)
Test(1, 1, 70, 8, 3, 8)
Test(1, 1, 70, 8, 3, 9)
Test(1, 1, 70, 8, 3, 10)
Test(1, 1, 70, 8, 3, 11)
print "--------------------------8^"
#Hardness = 9
Test(1, 1, 70, 9, 3, 7)
Test(1, 1, 70, 9, 3, 8)
Test(1, 1, 70, 9, 3, 9)
Test(1, 1, 70, 9, 3, 10)
Test(1, 1, 70, 9, 3, 11)
print "--------------------------9^"


TP: 6216, FP: 2525, FN: 2519
Pr: 71.11%, Re: 71.16%, F1 = 71.14%.


TP: 8734, FP: 4564, FN: 2171
Pr: 65.68%, Re: 80.09%, F1 = 72.17%.


TP: 11525, FP: 7551, FN: 1880
Pr: 60.42%, Re: 85.98%, F1 = 70.96%.


TP: 14352, FP: 11309, FN: 1677
Pr: 55.93%, Re: 89.54%, F1 = 68.85%.


TP: 17628, FP: 16228, FN: 1507
Pr: 52.07%, Re: 92.12%, F1 = 66.53%.

--------------------------8^

TP: 4208, FP: 1250, FN: 3131
Pr: 77.10%, Re: 57.34%, F1 = 65.77%.


TP: 6023, FP: 2289, FN: 2707
Pr: 72.46%, Re: 68.99%, F1 = 70.68%.


TP: 8021, FP: 3832, FN: 2396
Pr: 67.67%, Re: 77.00%, F1 = 72.03%.


TP: 10015, FP: 5821, FN: 2155
Pr: 63.24%, Re: 82.29%, F1 = 71.52%.


TP: 12378, FP: 8565, FN: 1912
Pr: 59.10%, Re: 86.62%, F1 = 70.26%.

--------------------------9^
